# 数据处理 Preprocess the CORD-19 Data

In [1]:
import re
import os

def add_paper(file_path):
    body_txt = ''
    with open(file_path,'r') as f:
        paper = json.load(f)
    for i in paper['abstract']:
        body_txt = body_txt + i['text']
        
    regex1 = r'\(.*?\)'
    pattern1 = re.compile(regex1)            
    body_txt = re.sub(pattern1,'',body_txt)
    regex2 = r'\[.*?\]'
    pattern2 = re.compile(regex2)            
    body_txt = re.sub(pattern2,'',body_txt)
    regex3 = r'The copyright holder for this preprint.*?preprint'
    pattern3 = re.compile(regex3)            
    body_txt = re.sub(pattern3,'',body_txt)
    
    body_txt = body_txt + '\n\n\n'
    return body_txt

body_txt = ''
paths = ['E:/COVID-19/2020-03-13/biorxiv_medrxiv/biorxiv_medrxiv',
         'E:/COVID-19/2020-03-13/comm_use_subset/comm_use_subset',
         'E:/COVID-19/2020-03-13/noncomm_use_subset/noncomm_use_subset',
         'E:/COVID-19/2020-03-13/pmc_custom_license/pmc_custom_license']

# paths = ['E:/COVID-19/2020-03-13/pmc_custom_license/pmc_custom_license']
for path in paths:
    print('start')
    files = os.listdir(path)
    for file in files:
        body_txt = body_txt + add_paper(path+'/'+file)
    print(path)

with open('covid19_paper.txt','w', encoding='utf-8') as f:
    f.write(body_txt)

start
E:/COVID-19/2020-03-13/biorxiv_medrxiv/biorxiv_medrxiv
start
E:/COVID-19/2020-03-13/comm_use_subset/comm_use_subset
start
E:/COVID-19/2020-03-13/noncomm_use_subset/noncomm_use_subset
start
E:/COVID-19/2020-03-13/pmc_custom_license/pmc_custom_license


In [2]:
import os
num = 0
paths = ['E:/COVID-19/2020-03-13/biorxiv_medrxiv/biorxiv_medrxiv',
         'E:/COVID-19/2020-03-13/comm_use_subset/comm_use_subset',
         'E:/COVID-19/2020-03-13/noncomm_use_subset/noncomm_use_subset',
         'E:/COVID-19/2020-03-13/pmc_custom_license/pmc_custom_license']

for path in paths:
    files = os.listdir(path)
    num = num + len(files)

print(f"There are {num} papers in total")

There are 13202 papers in total


# 选出包含risk factor的句子 
## Select the Sentences Containing the Risk Factors We Chose

In [6]:
import re
import numpy as np
risk_factors = ['older adult','elder age','pregnan','lung disease','heart disease',
                'cardiovascular disease','coronary disease',
               'liver disease','nervous system disease','chronic kidney disease',
               'chronic respiratory disease','coagulation dysfunction',
               'diabetes','HIV','cancer','asthma','sepsis','hypertension',
               'arrhythmia','myocardial infarction','chronic renal failure',
               'dyspnea','high fever','neutrophilia','lymphocytopenia',
               'methylprednisolone','leukocytosis','lactate dehydrogenase','LDH',
               'plasma urea','serum creatinine','IL-6','CD4','CD3','CD8',
               'high-sensitivity cardiac troponin','high-sensitivity C-reactive protein',
               'd-dimer','D-dime','aspartate aminotransferase','alanine aminotransferase',
               'medical resource','socioeconomics']
def findsen(name):
    with open('covid19_abstract.txt','r',encoding='utf-8') as f:
        text = f.read()
    return re.findall(r'[^.]*?{}[^。]*?\.(?![0-9])'.format(name), text)
a = []
for i in range(len(risk_factors)):
    sen = findsen(risk_factors[i])
    a.append([])
    a[i] = sen
# a = np.array(a)
# np.save('risk_factor_sen.npy', a)
# a = np.load('risk_factor_sen.npy')
# a = a.tolist()

In [1]:
import re
import numpy as np
name = 'older adult'
with open('covid19_part4.txt','r',encoding='utf-8') as f:
    text = f.read()
results = re.findall(r'[^.]*?{}[^.]*?\.(?![0-9])'.format(name), text)
for i,r in enumerate(results,1):
    print(i,r)

1  LRTI occurs less frequently, but young children, older adults, and persons who are immunosuppressed appear to be at higher risk for these types of infections    .
2 Unlike the other currently recognized human CoVs, HCoV-229E, HCoV-OC43, HCoV-NL63, and HKU1, which usually cause mild upper respiratory tract infections and occasionally pneumonia in older adults, neonates, and immunocompromised patients     , SARS-CoV causes severe febrile lower respiratory tract illness that leads to pneumonia and acute respiratory distress  .
3  Such a finding is hardly surprising in view of the high infection rates which have been observed in unselected older adults.
4  Our findings that older adults are more aware than younger persons has also been found in other settings  as has the increased public health awareness in more educated groups .
5  We also detected MERS-CoV RNA in a higher proportion of specimens from younger than from older adult dromedary camels, consistent with findings of previous 

# 过滤，直接含有CoV和coronavirus的句子
## Get the Sentences that contain 'CoV' and 'coronavirus'

In [4]:
import numpy as np
import pandas as pd
a = np.load('risk_factor_sen.npy')
a = a.tolist()
sen_contain_cov=[]
for i in a:
    for j in i:
        if 'CoV' in j or 'coronavirus' in j:
            sen_contain_cov.append(j)
print(len(sen_contain_cov))
labels = [0]*len(sen_contain_cov)
df = pd.DataFrame({"sentence":sen_contain_cov, "labels":labels})
df.to_excel('sen_contain_cov.xls')

2170


# 得到三个训练样本集和测试集
## Get 3 train sets and test set

In [37]:
from pandas import DataFrame
import pandas as pd
df1 = pd.read_excel('data1.xlsx')
df2 = pd.read_excel('sen_irrelevant.xls')
p_sent = df1[df1['labels']==1].reset_index()
n1_sent = df1[df1['labels']==0].reset_index()
n2_sent = df2


In [38]:
data2 = pd.concat([p_sent[0:400], n2_sent]).reset_index()
data3 = pd.concat([p_sent[0:400],n1_sent[0:500],n2_sent[0:500]]).reset_index()
data2.to_excel('data2.xlsx')
data3.to_excel('data3.xlsx')

In [39]:
data_test = pd.concat([p_sent[400:500],n1_sent[500:600],n2_sent[500:600]]).reset_index()
data_test.to_excel('data_test.xlsx')

In [40]:
n2_sent['labels'] = 2
data4 = pd.concat([p_sent[0:400],n1_sent[0:500],n2_sent[0:500]]).reset_index()
data4.to_excel('data4.xlsx')

In [2]:
paths = ['E:/COVID-19/2020-03-13/biorxiv_medrxiv/biorxiv_medrxiv',
         'E:/COVID-19/2020-03-13/comm_use_subset/comm_use_subset',
         'E:/COVID-19/2020-03-13/noncomm_use_subset/noncomm_use_subset',
         'E:/COVID-19/2020-03-13/pmc_custom_license/pmc_custom_license']
total_paper_num = 0
for path in paths:
    files = os.listdir(path)
    total_paper_num = total_paper_num + len(files)
total_paper_num

13202

# 得到处理好的所有句子
## Get all sentences in fine version

In [15]:
import json
import re
import pandas as pd
import os

def clear_sent(sent):
    i = 0
    while sent[i]==' ' or sent[i] in ['0','1','2','3','4','5','6','7','8','9']:
        i = i + 1
    sent = sent[i:]
    return sent

def add_paper(file_path):
    body_txt = ''
    with open(file_path,'r') as f:
        paper = json.load(f)
    for i in paper['body_text']:
        if 'preprint' not in i['text'] and 'rights reserved' not in i['text']:
            body_txt = body_txt + i['text']
    for i in paper['abstract']:
        if 'preprint' not in i['text'] and 'rights reserved' not in i['text']:
            body_txt = body_txt + i['text']
        
    regex1 = r'\(.*?\)'
    pattern1 = re.compile(regex1)            
    body_txt = re.sub(pattern1,'',body_txt)
    regex2 = r'\[.*?\]'
    pattern2 = re.compile(regex2)            
    body_txt = re.sub(pattern2,'',body_txt)
    sentences = re.findall(r'[^.]*?\.(?![0-9])', body_txt)
    sentences = [clear_sent(sent) for sent in sentences]

    return sentences

paths = ['E:/COVID-19/2020-03-13/biorxiv_medrxiv/biorxiv_medrxiv',
         'E:/COVID-19/2020-03-13/comm_use_subset/comm_use_subset',
         'E:/COVID-19/2020-03-13/noncomm_use_subset/noncomm_use_subset',
         'E:/COVID-19/2020-03-13/pmc_custom_license/pmc_custom_license']

all_sentences = []
for path in paths:
    print('start')
    files = os.listdir(path)
    for file in files:
        all_sentences.extend(add_paper(path+'/'+file))



start
start
start
start


In [16]:
all_sent_fine_ver = []
for sent in all_sentences:
    if len(sent)>30:
        all_sent_fine_ver.append(sent)
print(len(all_sent_fine_ver))
df = pd.DataFrame({'sentence':all_sent_fine_ver})
df.to_excel('all_sentences.xlsx')


2262149


In [21]:
df.sentence[11394]

'The domain of integration for each parameter is all that remains to be solved.'